In [ ]:

from scipy import signal
from scipy.integrate import simps
import numpy as np
from fooof import FOOOF
import matplotlib.pyplot as plt
import pickle

def harmonics_removal(signal, fs, harmonics, dftbandwidth=1, dftneighbourwidth=2):
    """
    Removes beta harmonics in a signal via spectrum interpolation.

    Parameters:
    - signal: 1D numpy array, the input signal
    - fs: float, sampling rate in Hz
    - harmonics: list of floats, harmonics frequencies in Hz (e.g., [50, 100, 150])
    - dftbandwidth: float, half bandwidth of harmonics frequency bands in Hz (default 1)
    - dftneighbourwidth: float, width of neighbouring frequencies in Hz (default 2)

    Returns:
    - cleaned_signal: 1D numpy array, the signal withouth the indicated beta harmonics    """
    # FFT of the signal
    N = len(signal)
    freqs = np.fft.fftfreq(N, 1/fs)
    signal_fft = np.fft.fft(signal)

    # Helper function to get indices of frequency bins
    def get_freq_indices(freq, bandwidth, fs, N):
        
        return np.where((freqs >= (freq - bandwidth)) & (freqs <= (freq + bandwidth)))[0]

    # Process each harmonic
    for f in harmonics:
        harmonics_indices = get_freq_indices(f, dftbandwidth, fs, N)
       
        harmonics_indices = np.concatenate((harmonics_indices, get_freq_indices(-f, dftbandwidth, fs, N)))
       
        for harmonics_index in harmonics_indices:
            # Find neighbouring indices
            lower_bound = f - dftneighbourwidth - dftbandwidth
            upper_bound = f + dftneighbourwidth + dftbandwidth
            neighbours = np.where((freqs >= lower_bound) & (freqs <= upper_bound) & 
                                  ((freqs < (f - dftbandwidth)) | (freqs > (f + dftbandwidth))))[0]
            
            # Compute the mean amplitude of neighbouring bins
            if len(neighbours) > 1:
                neighbour_freqs = freqs[neighbours]
          
            
                neighbour_amplitudes = np.abs(signal_fft[neighbours])
                
                interpolated_amplitude = np.mean(neighbour_amplitudes)
                original_phase = np.angle(signal_fft[harmonics_index])
                # Replace the amplitude of the harmonics frequency bin by the interpolated value
                signal_fft[harmonics_index] = interpolated_amplitude * np.exp(1j * original_phase)
    # Inverse FFT to get the cleaned signal

    cleaned_signal = np.fft.ifft(signal_fft).real
    
    return cleaned_signal

#Center of gravity method for computing central frequency
def  cog(f,pxx,f1,f2):
    prod=f*pxx
    cog=abs(simps(prod[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)])/simps(pxx[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)]))
    return cog

#Aperiodic function that will be fitted and removed for the computation of the spectral power

def aper(f,offset,exp):
     return 10**offset/(f**exp)


   

   

Identification of most relevant connections for GPe-TI gamma oscillations

In [ ]:

result = pickle.load(open("disc.p", "rb"))

# Sampling frequency and parameters for Welch's method
fs = 1000
nparseg = 1000

# Frequency range for focus (GPe-TI gamma oscillations)
fmin_f = 50
fmax_f = 150

# Dictionary to store results for each disconnected projection
pow = {}

# Loop through all keys in the result dictionary
for disc in result:
    gamma = []  # List to store gamma power for each disconnected projection
    fr = []     # List to store firing rates


    # Iterate through the 5 simulations for each disconnected projection
    for i in range(5):
        hist_GPe_TI, hist_D2 = result[disc][i]

        # Compute the Welch's Power Spectral Density
        f, pxx = signal.welch(hist_GPe_TI, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                              nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Calculate the mean firing rate
        fr.append(np.mean(hist_GPe_TI / 780 * 1000))

        # Center of gravity for computing beta peak
        f_beta = cog(f, pxx, 10, 30)

        # Removing the harmonics 
        #Disconnecting self-inhibition enhance beta activity, hence more harmonics should be removed
        if disc == 'D2-D2' or disc == 'GPe-TI-GPe-TI':
            harmonics = np.arange(2, 10) * f_beta
            hist_GPe_TI = harmonics_removal(hist_GPe_TI, fs, harmonics, 6, 3)
            f, pxx = signal.welch(hist_GPe_TI, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                                  nfft=max(30000, nparseg), scaling='density', window='hamming')
        #These are the connections that destroy the beta: without them there are no harmonics
        elif disc not in ['FSN-D2', 'D2-GPe-TI', 'GPe-TI-FSN', 'GPe-TI-STN', 'STN-GPe-TI', 'GPe-TA-FSN']:
            harmonics = np.arange(2, 5) * f_beta
            hist_GPe_TI = harmonics_removal(hist_GPe_TI, fs, harmonics, 3, 3)
            f, pxx = signal.welch(hist_GPe_TI, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                                  nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Fit a FOOOF model to extract aperiodic component
        fm = FOOOF(max_n_peaks=3, peak_width_limits=[15, 50], verbose=False)
        fm.fit(f, pxx, freq_range=[5, 500])

        # Subtract aperiodic fit from PSD
        pxx[f > 5] = pxx[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Compute center of gravity for gamma frequency range
        f_cog = cog(f, pxx, fmin_f, fmax_f)

        # Calculate gamma power in a 40 Hz band around f_cog
        fmin = f_cog - 20
        fmax = f_cog + 20
        gamma_power = simps(pxx[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)])
        gamma_power = gamma_power / (fmax - fmin)

        # Simulate Poissonian activity for the threshold 
        if i == 4:
            gamma_power_pois = []
            for n in range(100):
                pois = np.random.binomial(780, np.mean(np.array(fr) / 1000), len(hist_GPe_TI))
                f_poisson, Pxx__poisson = signal.welch(pois, fs, nperseg=nparseg, noverlap=int(nparseg / 2),
                                                       nfft=max(30000, nparseg), scaling='density', window='hamming')
                pw = simps(Pxx__poisson[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)])
                pw = pw / (fmax - fmin)
                gamma_power_pois.append(pw)

            # Threshold for gamma activity
            treshold_gamma = np.mean(np.array(gamma_power_pois))

        gamma.append(gamma_power)

    # Store computed results for the current disconnected projection
    pow[disc] = (np.mean(np.array(gamma)), np.std(np.array(gamma)), 
                 np.mean(np.array(fr)), np.std(np.array(fr)), treshold_gamma)

# Save and finalize plots
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=30)

err_or = pow['original'][1] / pow['original'][0]
gamma_or = pow['original'][0]
fir_or = pow['original'][2]

# Process results to compute the ratio of gamma power with the original case. The error is obtained with standard propagation rules

final = {}
for disc in pow:
    final[disc] = (np.sqrt(((pow[disc][1] / pow[disc][0])**2 + err_or**2)) * (pow[disc][0] / gamma_or), 
                   pow[disc][0] / gamma_or, pow[disc][4] / gamma_or)

labels = list(final.keys())
x = range(len(labels))
means = [value[1] for value in final.values()]
errors = [value[0] for value in final.values()]
thresh = [value[2] for value in final.values()]

# Create error bar plot
plt.figure(figsize=(12, 10))
plt.errorbar(x, means, yerr=errors, fmt='.', markersize=18, capsize=10, color='blue')
plt.ylabel('$R_{\\gamma}$', labelpad=12, fontsize=30)
plt.axhline(1, linestyle='--', color='red', lw=4)
plt.plot(x, thresh, color='black', linestyle='--', lw=4)
plt.xticks(x, labels, rotation='vertical')
plt.tight_layout()
plt.show()



Comparison of the power spectral densities with and without GPe-TI self-inhibition 

In [ ]:
#require the first cell to be run
result = pickle.load(open("disc.p", "rb"))
from cycler import cycler
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=26)
plt.rc('ytick', labelsize=25)
fig=plt.figure(figsize=(16,12))

color_cycler = cycler(color=[ 'blue','red'])
plt.rc('axes', prop_cycle=color_cycler)
for disc in result.keys():
   #plotting just original case and the one with GPe-TI-GPe-TI disconnected
    if disc=='original' or disc=='GPe-TI-GPe-TI':
        pxx_final=[]
        t_s=1

        nparseg = int(1000/t_s)

        fs = 1000/t_s
        n_trials=5  
  
        for i in np.arange(0,n_trials,1):
     
            hist_GPe_TI, hist_D2=result[disc][i]
            f, pxx = signal.welch(hist_GPe_TI, fs, nperseg=nparseg, noverlap=int(nparseg/2),nfft=max(30000,nparseg), scaling='density', window='hamming')
            #computing psds (for frequncy below 250 Hz)
            pxx =pxx [f>5]
            f=f[f>5]
            plt.xlim(0,250)                    
            pxx_final.append(pxx)
         

           
        if disc=='GPe-TI-GPe-TI':
          plt.plot(f,np.mean(np.array(pxx_final),axis=0),linewidth=2.5,label="GPe-TI to GPe-TI \ndisconnected")
        else:
          plt.plot(f,np.mean(np.array(pxx_final),axis=0),linewidth=2.5,label="Original")
        plt.fill_between(f,np.mean(pxx_final,axis=0)-np.std(pxx_final,axis=0),np.mean(pxx_final,axis=0)+np.std(pxx_final,axis=0),alpha=0.5)
 
       
        plt.yscale('log')
        plt.xlabel("Frequency [Hz]",labelpad=10)
        plt.ylabel('PSD [a.u.]')
        legend=plt.legend(framealpha=0)
        legend=plt.legend(loc='upper right',framealpha=0)
        for line in legend.get_lines():
          line.set_linewidth(5)  


plt.show()

# D2

Identification of most relevant connections for D2 gamma oscillations

In [ ]:

result = pickle.load(open("disc.p", "rb"))

# Sampling frequency and parameters for Welch's method
fs = 1000
nparseg = 1000

# Frequency range for focus (for D2 gamma oscillations)
fmin_f = 40
fmax_f = 120

# Dictionary to store results for each disconnected projection
pow = {}

# Loop through all keys in the result dictionary
for disc in result:
    gamma = []  # List to store gamma power for each disconnected projection
    fr = []     # List to store firing rates
    

    # Iterate through the 5 simulations of each disconnected projection
    for i in range(5):
        hist_GPe_TI, hist_D2 = result[disc][i]

        # Compute the Welch's Power Spectral Density
        f, pxx = signal.welch(hist_D2, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                              nfft=max(30000, nparseg), scaling='density', window='hamming')

        
        # Calculate the mean firing rate
        fr.append(np.mean(hist_D2 / 6000 * 1000))

        # Center of gravity for beta frequency range (10–30 Hz)
        f_beta = cog(f, pxx, 10, 30)

        # Removing the harmonics 
        #Disconnecting self-inhibition enhances beta activity, hence more harmonics should be removed
        if disc == 'D2-D2' or disc == 'GPe-TI-GPe-TI':
            harmonics = np.arange(2, 10) * f_beta
            hist_D2 = harmonics_removal(hist_D2, fs, harmonics, 5, 3)
            f, pxx = signal.welch(hist_D2, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                                  nfft=max(30000, nparseg), scaling='density', window='hamming')
        #These are the connections that destroy the beta: without them there are no harmonics
        elif disc not in ['FSN-D2', 'D2-GPe-TI', 'GPe-TI-FSN', 'GPe-TI-STN', 'STN-GPe-TI', 'GPe-TA-FSN']:

            harmonics = np.arange(2, 5) * f_beta
            hist_D2 = harmonics_removal(hist_D2, fs, harmonics, 5, 3)
            f, pxx = signal.welch(hist_D2, fs, nperseg=nparseg, noverlap=int(nparseg / 2), 
                                  nfft=max(30000, nparseg), scaling='density', window='hamming')
     
      
        # Fit a FOOOF model to extract aperiodic component
        fm = FOOOF(max_n_peaks=2, peak_width_limits=[15, 50], verbose=False)
        fm.fit(f, pxx, freq_range=[5, 500])

        # Subtract aperiodic fit from PSD
        pxx[f > 5] = pxx[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Compute center of gravity for gamma frequency range
        f_cog = cog(f, abs(pxx), fmin_f, fmax_f)

        # Calculate gamma power in a 40 Hz band around f_cog
        fmin = f_cog - 20
        fmax = f_cog + 20
  
        gamma_power = simps(pxx[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)])
        gamma_power = gamma_power / (fmax - fmin)

        # Simulate Poissonian activity for the threshold 
        if i == 4:
            gamma_power_pois = []
            for n in range(100):
                pois = np.random.binomial(6000, np.mean(np.array(fr) / 1000), len(hist_D2))
                f_poisson, Pxx__poisson = signal.welch(pois, fs, nperseg=nparseg, noverlap=int(nparseg / 2),
                                                       nfft=max(30000, nparseg), scaling='density', window='hamming')
                pw = simps(Pxx__poisson[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)])
                pw = pw / (fmax - fmin)
                gamma_power_pois.append(pw)

            # Threshold for gamma activity
            treshold_gamma = np.mean(np.array(gamma_power_pois))

        gamma.append(gamma_power)

    # Store computed results for the current disconnected projection
    pow[disc] = (np.mean(np.array(gamma)), np.std(np.array(gamma)), 
                 np.mean(np.array(fr)), np.std(np.array(fr)), treshold_gamma)

# Save and finalize plots
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=30)

err_or = pow['original'][1] / pow['original'][0]
gamma_or = pow['original'][0]
fir_or = pow['original'][2]

# Process results to compute the ratio of gamma power with the original case. The error is obtained with standard propagation rules

final = {}
for disc in pow:
    final[disc] = (abs(np.sqrt(((pow[disc][1] / pow[disc][0])**2 + err_or**2)) * (pow[disc][0] / gamma_or)), 
                   pow[disc][0] / gamma_or, pow[disc][4] / gamma_or)

labels = list(final.keys())
x = range(len(labels))
means = [value[1] for value in final.values()]
errors = [value[0] for value in final.values()]
thresh = [value[2] for value in final.values()]

# Create error bar plot
plt.figure(figsize=(12, 10))
plt.errorbar(x, means, yerr=errors, fmt='.', markersize=18, capsize=10, color='blue')
plt.ylabel('$R_{\\gamma}$', labelpad=12, fontsize=30)
plt.axhline(1, linestyle='--', color='red', lw=4)
plt.plot(x, thresh, color='black', linestyle='--', lw=4)
plt.xticks(x, labels, rotation='vertical')
plt.tight_layout()
plt.show()



Comparison of the power spectral densities with and without D2 self-inhibition 

In [ ]:
#require the first cell to be run
result = pickle.load(open("disc.p", "rb"))
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=26)
plt.rc('ytick', labelsize=25)
fig=plt.figure(figsize=(16,12))
from cycler import cycler


color_cycler = cycler(color=[ 'blue','red'])
plt.rc('axes', prop_cycle=color_cycler)
for disc in result.keys():
   
    if disc=='original' or disc=='D2-D2':
        pxx_final=[]
          
        t_s=1

        nparseg = int(1000/t_s)

        fs = 1000/t_s
        n_trials=5  
  
        for i in np.arange(0,n_trials,1):
     
            hist_GPe_TI, hist_D2=result[disc][i]
            f, pxx = signal.welch(hist_D2, fs, nperseg=nparseg, noverlap=int(nparseg/2),nfft=max(30000,nparseg), scaling='density', window='hamming')
            #computing psds (for frequncy below 250 Hz)
            pxx =pxx [f>5]
            f=f[f>5]
            plt.xlim(0,250)                    
            pxx_final.append(pxx)
         

           
        if disc=='D2-D2':
          plt.plot(f,np.mean(np.array(pxx_final),axis=0),linewidth=2.5,label="D2 to D2 \ndisconnected")
        else:
          plt.plot(f,np.mean(np.array(pxx_final),axis=0),linewidth=2.5,label="Original")
        plt.fill_between(f,np.mean(pxx_final,axis=0)-np.std(pxx_final,axis=0),np.mean(pxx_final,axis=0)+np.std(pxx_final,axis=0),alpha=0.5)
 
       
        plt.yscale('log')
        plt.xlabel("Frequency [Hz]",labelpad=10)
        plt.ylabel('PSD [a.u.]')
        legend=plt.legend(framealpha=0)
        legend=plt.legend(loc='upper right',framealpha=0)
        for line in legend.get_lines():
          line.set_linewidth(5)  


plt.show()